# 23.2 MARL (MADDPG)
> 前文已经介绍了 **多智能体强化学习的基本求解范式：完全中心化 与 完全去中心化**。本节来介绍一种比较经典且效果不错的**进阶范式：中心化训练去中心化执行（centralized training with decentralized execution，CTDE）**：
> - **中心化训练**：在训练的时候使用一些单个智能体看不到的 **全局信息** 而以达到更好的训练效果。
> - **去中心化执行**：在执行时不使用 **全局信息**，每个智能体完全根据自己的策略执行动作。

**CTDE** 可以类比成一个足球队的训练和比赛过程：在训练时，11 个球员可以直接获得教练的指导从而完成球队的整体配合，而教练本身掌握着比赛全局信息，教练的指导也是从整支队、整场比赛的角度进行的；而训练好的 11 个球员在上场比赛时，则根据场上的实时情况直接做出决策，不再有教练的指导。
> 所以 **CTDE** 能够在训练时有效地利用全局信息以达到 **更好且更稳定的训练效果**，同时在进行策略模型推断时可以仅利用局部信息，使得算法**具有一定的扩展性**。

> **CTDE** 算法主要分为两种：
> - 一种是基于值函数的方法，例如 **VDN，QMIX** 
> - 一种是基于 Actor-Critic 的方法，例如 **MADDPG，COMA** 

> 本节介绍 **MADDPG** 算法 （[Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments](https://arxiv.org/abs/1706.02275)）

## 23.2.1 MADDPG 算法原理
- 学习本节内容之前，可以先复习之前的 [DDPG 算法](../RL_Classic_Algorithms/16.DDPG(Pendulum-v1).ipynb) ，梯度为：
$$\nabla_\theta J(\mu_\theta)=\mathbb{E}_{s\sim\nu^{\pi_\beta}}
\begin{bmatrix}
\nabla_\theta\mu_\theta(s)\nabla_aQ_\omega^{\mu_\theta}(s,a)|_{a=\mu_\theta(s)}
\end{bmatrix}$$
> 与 **‘纯DDPG’** 不同，在 **MADDPG** 中 **所有智能体共享一个中心化的 Critic 网络**，该 Critic 网络在训练的过程中同时对每个智能体的 Actor 网络给出指导；而执行时，**每个智能体的 Actor 网络** 则是完全独立做出行动，即去中心化地执行：

> ![Overview_of_MADDPG_AC](./Illustrations/Overview_of_MADDPG_AC.png)

考虑现在有$N$个连续的策略$\mu_{\theta_i}$，有梯度公式：
$$\nabla_{\theta_i}J(\mu_i)=\mathbb{E}_{\mathbf{x}\sim\mathcal{D}}[\nabla_{\theta_i}\mu_i(o_i)\nabla_{a_i}Q_i^\mu(\mathbf{x},a_1,\ldots,a_N)|_{a_i=\mu_i(o_i)}]$$
- 对于每个智能体 $i$，其动作空间为$A_i$，观测空间为$O_i$
- $\mathbf{x}=(o_{1},\ldots,o_{N})$ 包含了所有智能体的观测
- $\mathcal{D}$ 表示存储数据的经验回放池，它存储的每一个数据为 $(\mathbf{x},\mathbf{x}^{\prime},a_1,\ldots,a_N,r_1,\ldots,r_N)$
- 可见，有一个 **Critic** $Q_w$，$N$个 **Actor** $\mu_\theta^i$

此时在 **MADDPG** 中，**中心化的动作价值网络$Q$** 更新的损失函数变为：
$$\mathcal{L}(\omega_i)=\mathbb{E}_{\mathbf{x},a,r,\mathbf{x}^{\prime}}[(Q_i^\mu(\mathbf{x},a_1,\ldots,a_N)-y)^2]$$
$$y=r_i+\gamma Q_i^{\mu^{\prime}}(\mathbf{x}^{\prime},a_1^{\prime},\ldots,a_N^{\prime})|_{a_j^{\prime}=\mu_j^{\prime}(o_j)}$$

## 23.2.2 MADDPG 代码实践

导入基本库

In [13]:
import numpy as np
import random

import torch
import torch.nn.functional as F

### 1. 关于环境库
同样需新建一个虚拟环境（防止软件包冲突），下载 **MPE2库**（[使用文档](https://mpe2.farama.org/environments/simple_adversary/)），使用其中的 **多智能体粒子环境（multiagent particles environment，MPE）**：
(如果图片加载异常，请在浏览器中打开 Notebook)
![mpe2_simple_adversary](./Illustrations/simple_adversary.gif)

- 

***测试：***

In [14]:
from mpe2 import simple_adversary_v3

In [15]:
env = simple_adversary_v3.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

### 2. 采样动作不可导问题：离散采样操作不可导，这会导致无法使用反向传播来优化模型（[复习SAC中使用的重参数化技巧（reparameterization trick）](../RL_Classic_Algorithms/17.1_SAC算法原理.ipynb)）
> **DDPG** 算法通过 **确定性策略** 针对 **连续动作空间**，使智能体的动作对于其 **策略参数$\mu_\theta$** 可导
> 但 **MPE** 环境中的每个智能体的 **动作空间是离散的**，虽然依旧是 **确定性策略**，但是需要对网络输出进行 **离散采样**，这种采样是 **不可导的**，此时就需要运用之前的 **重参数化方法（这里要用的是 Gumbel-Softmax 技巧）** 让离散分布的采样可导：

**Gumbel-Softmax 变换：**
1. Gumbel 随机变量的采样（重参数因子$g_i$）
$$g_i=-\log(-\log u),u\sim\mathrm{Uniform}(0,1)$$
2. 加入 Gumbel 噪声并进行 softmax
$$\tilde{y}_i=\frac{\exp\left(\frac{\log(\pi_i)+g_i}{\tau}\right)}{\sum_j\exp\left(\frac{\log(\pi_j)+g_j}{\tau}\right)}$$
- $\tau$ 是温度参数，用来控制离散度，温度越低，近似越接近离散采样
- 引入 Gumbel 随机变量和温度参数$\tau$，得到一个平滑的概率分布，使得模型的输出变得可微
- 将原本的离散采样过程（不可导）转化为连续的近似，这样，就能利用反向传播来优化模型


#### Gumbel Softmax 采样的相关函数：

In [16]:
def onehot_from_logits(logits, eps=0.01):
    """ 独热（one-hot）离散化 """
    # 生成最优动作的独热形式
    argmax_acs = (logits == logits.max(1, keepdim=True)[0]).float()
    # 生成随机动作的独热形式
    rand_acs = torch.autograd.Variable(torch.eye(logits.shape[1])[[
        np.random.choice(range(logits.shape[1]), size=logits.shape[0])
    ]], requires_grad=False).to(logits.device)
    
    # 通过epsilon-贪婪算法来选择用哪个动作
    return torch.stack([
        argmax_acs[i] if r > eps else rand_acs[i]
        for i, r in enumerate(torch.rand(logits.shape[0]))
    ])


def sample_gumbel(shape, eps=1e-20, tens_type=torch.FloatTensor):
    """ 1.Gumbel随机变量的采样 """
    U = torch.autograd.Variable(tens_type(*shape).uniform_(), requires_grad=False)
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature):
    """ 2.加入Gumbel噪声并进行softmax """
    y = logits + sample_gumbel(logits.shape, tens_type=type(logits.data)).to(logits.device)
    return F.softmax(y / temperature, dim=1)


def gumbel_softmax(logits, temperature=1.0):
    """ 从Gumbel-Softmax分布中采样,并进行离散化 """
    y = gumbel_softmax_sample(logits, temperature)
    y_hard = onehot_from_logits(y)
    y = (y_hard.to(logits.device) - y).detach() + y  # 使硬选择部分y_hard与模型的训练过程断开连接,但又保证在训练中通过软选择y反传梯度对模型进行优化
    return y  # 虽然返回的是独热量y_hard,但是它的梯度是y,既能够得到一个与环境交互的离散动作,又可以正确地反传梯度

### 3. 每个智能体：单智能体 DDPG


### 4. MADDPG 类：维护每个智能体

### 5. 测试与训练